In [3]:
import pandas as pd



pd.reset_option('display.max_colwidth')

data1 = pd.read_csv('../data/appartements1.csv')
data2 = pd.read_csv('../data/appartements2.csv')
appartements=pd.concat([data1,data2],axis=0,ignore_index=True)
df =appartements.copy()



In [4]:
import sklearn
print(sklearn.__version__)

1.4.0


In [5]:
from sklearn.model_selection import train_test_split
from preprocessing import Preprocessing 
p =Preprocessing()
X,y,vectorize =p.processing(df)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=0)


c:\Users\Admin\Desktop\hp\Projects\ML-project\processing\preprocessing.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['etat'] =df['etat'].str.lower().str.replace(' ','')
c:\Users\Admin\Desktop\hp\Projects\ML-project\processing\preprocessing.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['autre'] =df['autre'].str.lower()
c:\Users\Admin\Desktop\hp\Projects\ML-project\processing\preprocessing.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

#### RandomForest

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
randomForestRegressor = make_pipeline(RobustScaler(), RandomForestRegressor(max_features=10,
                                                                            min_samples_leaf= 1,
                                                                            min_samples_split= 2,
                                                                            n_estimators=200,
                                                                        
                                                                            ))


In [7]:
randomForestRegressor.fit(X_train,y_train)

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('randomforestregressor',
                 RandomForestRegressor(max_features=10, n_estimators=200))])

In [8]:
import pickle
file =open('../Models/RandomForest.pkl','wb')
pickle.dump(randomForestRegressor,file)

#### GradientBoosting

In [9]:
gradientBoosting = make_pipeline(RobustScaler(), GradientBoostingRegressor(learning_rate=0.06,
                                                                                 max_depth=7,
                                                                                 n_estimators=400,
                                                                                
                                                                            ))


In [10]:
gradientBoosting.fit(X_train,y_train)

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('gradientboostingregressor',
                 GradientBoostingRegressor(learning_rate=0.06, max_depth=7,
                                           n_estimators=400))])

In [11]:
import pickle
file =open('../Models/grdBoosting.pkl','wb')
pickle.dump(gradientBoosting,file)

#### Stacking

In [12]:
import pickle
from sklearn.linear_model import Lasso
from sklearn.ensemble import StackingRegressor

##--- import the best model of randomForest
file =open('../Models/RandomForest.pkl','rb')
best_random_forest=pickle.load(file)
    
##--- import the best model of GradientBoosting
file =open('../Models/grdBoosting.pkl','rb')
best_gradient_boosting=pickle.load(file)


base_regressors = [
    ('random_forest',best_random_forest),
    ('gradient_boosting', best_gradient_boosting),
]
final_estimator = Lasso(alpha=0.7)  # Adjust alpha 

stacking_regressor = StackingRegressor(
    estimators=base_regressors,
    final_estimator=final_estimator,n_jobs=-1
)

# Train the StackingRegressor on the training data
stacking_regressor.fit(X_train, y_train)




StackingRegressor(estimators=[('random_forest',
                               Pipeline(steps=[('robustscaler', RobustScaler()),
                                               ('randomforestregressor',
                                                RandomForestRegressor(max_features=10,
                                                                      n_estimators=200))])),
                              ('gradient_boosting',
                               Pipeline(steps=[('robustscaler', RobustScaler()),
                                               ('gradientboostingregressor',
                                                GradientBoostingRegressor(learning_rate=0.06,
                                                                          max_depth=7,
                                                                          n_estimators=400))]))],
                  final_estimator=Lasso(alpha=0.7), n_jobs=-1)

In [14]:
file = open("../Models/Stacking.pkl",'wb')
pickle.dump(stacking_regressor,file=file)